Below: For all pick combinations

In [85]:
class Node:
    def __init__(self, assigned_picks=None, first_possible=True, second_possible=True):
        self.child_nodes = []
        self.assigned_picks = assigned_picks if assigned_picks is not None else []
        self.first_possible = first_possible
        self.second_possible = second_possible

    def get_copy(self):
        copy_node = Node(self.assigned_picks.copy(), self.first_possible, self.second_possible)
        return copy_node

    def __str__(self):
        picks_str = ", ".join(map(str, self.assigned_picks))
        possible_str = []
        if self.first_possible:
            possible_str.append("first")
        if self.second_possible:
            possible_str.append("second")
        return f"[{picks_str}] ({', '.join(possible_str)})"


def calculate_pick_combinations(parent_node, territory_count):
    if len(parent_node.assigned_picks) == territory_count:
        return

    first_still_possible = parent_node.first_possible
    second_still_possible = parent_node.second_possible
    current_depth = len(parent_node.assigned_picks) + 1

    own_picks = current_depth - 1

    # Player A is picking
    if first_still_possible:
        opponent_picks = current_depth - 1 if current_depth % 2 == 1 else current_depth
        max_pick_number = opponent_picks + 1 + own_picks
        highest_current_pick = parent_node.assigned_picks[-1] if current_depth != 1 else 0

        for i in range(highest_current_pick + 1, max_pick_number + 1):
            child_node = parent_node.get_copy()
            child_node.second_possible = False  # After Player A picks, Player B picks
            child_node.assigned_picks.append(i)
            parent_node.child_nodes.append(child_node)
            calculate_pick_combinations(child_node, territory_count)

    # Player B is picking
    if second_still_possible:
        opponent_picks = current_depth if current_depth % 2 == 1 else current_depth - 1
        max_pick_number = opponent_picks + 1 + own_picks
        highest_current_pick = parent_node.assigned_picks[-1] if current_depth != 1 else 0

        for i in range(highest_current_pick + 1, max_pick_number + 1):
            child_node = parent_node.get_copy()
            child_node.first_possible = False  # After Player B picks, Player A picks
            child_node.assigned_picks.append(i)
            parent_node.child_nodes.append(child_node)
            calculate_pick_combinations(child_node, territory_count)


def get_all_leaves(root):
    leaves = []

    if not root.child_nodes:
        leaves.append(root)
    else:
        for child in root.child_nodes:
            leaves.extend(get_all_leaves(child))

    return leaves


def print_combinations(root):
    leaf_nodes = get_all_leaves(root)
    seen_combinations = {}  # Dictionary to track combinations

    for node in leaf_nodes:
        picks_tuple = tuple(node.assigned_picks)  # Convert list to tuple to use as dict key
        if picks_tuple in seen_combinations:
            # If combination already seen, update first and second possible flags
            if node.first_possible:
                seen_combinations[picks_tuple].first_possible = True
            if node.second_possible:
                seen_combinations[picks_tuple].second_possible = True
        else:
            # New combination, add it to the dictionary
            seen_combinations[picks_tuple] = node

    # Now print all unique combinations
    for unique_node in seen_combinations.values():
        print(unique_node)


def main(territory_count=8):
    root = Node()
    calculate_pick_combinations(root, territory_count)
    print_combinations(root)


if __name__ == "__main__":
    main(territory_count=8)


[1, 2, 3, 4, 5, 6, 7, 8] (first, second)
[1, 2, 3, 4, 5, 6, 7, 9] (first, second)
[1, 2, 3, 4, 5, 6, 7, 10] (first, second)
[1, 2, 3, 4, 5, 6, 7, 11] (first, second)
[1, 2, 3, 4, 5, 6, 7, 12] (first, second)
[1, 2, 3, 4, 5, 6, 7, 13] (first, second)
[1, 2, 3, 4, 5, 6, 7, 14] (first, second)
[1, 2, 3, 4, 5, 6, 7, 15] (first, second)
[1, 2, 3, 4, 5, 6, 7, 16] (first)
[1, 2, 3, 4, 5, 6, 8, 9] (first, second)
[1, 2, 3, 4, 5, 6, 8, 10] (first, second)
[1, 2, 3, 4, 5, 6, 8, 11] (first, second)
[1, 2, 3, 4, 5, 6, 8, 12] (first, second)
[1, 2, 3, 4, 5, 6, 8, 13] (first, second)
[1, 2, 3, 4, 5, 6, 8, 14] (first, second)
[1, 2, 3, 4, 5, 6, 8, 15] (first, second)
[1, 2, 3, 4, 5, 6, 8, 16] (first)
[1, 2, 3, 4, 5, 6, 9, 10] (first, second)
[1, 2, 3, 4, 5, 6, 9, 11] (first, second)
[1, 2, 3, 4, 5, 6, 9, 12] (first, second)
[1, 2, 3, 4, 5, 6, 9, 13] (first, second)
[1, 2, 3, 4, 5, 6, 9, 14] (first, second)
[1, 2, 3, 4, 5, 6, 9, 15] (first, second)
[1, 2, 3, 4, 5, 6, 9, 16] (first)
[1, 2, 3, 4, 5, 6, 


Below: For pick combinations filtered based on top n

In [84]:
from collections import defaultdict

class Node:
    def __init__(self, assigned_picks=None, first_possible=True, second_possible=True):
        self.child_nodes = []
        self.assigned_picks = assigned_picks if assigned_picks is not None else []
        self.first_possible = first_possible
        self.second_possible = second_possible

    def get_copy(self):
        copy_node = Node(self.assigned_picks.copy(), self.first_possible, self.second_possible)
        return copy_node

    def __str__(self):
        picks_str = ", ".join(map(str, self.assigned_picks))
        possible_str = []
        if self.first_possible:
            possible_str.append("first")
        if self.second_possible:
            possible_str.append("second")
        return f"[{picks_str}] ({', '.join(possible_str)})"


def calculate_pick_combinations(parent_node, territory_count):
    if len(parent_node.assigned_picks) == territory_count:
        return

    first_still_possible = parent_node.first_possible
    second_still_possible = parent_node.second_possible
    current_depth = len(parent_node.assigned_picks) + 1

    own_picks = current_depth - 1

    # Player A is picking
    if first_still_possible:
        opponent_picks = current_depth - 1 if current_depth % 2 == 1 else current_depth
        max_pick_number = opponent_picks + 1 + own_picks
        highest_current_pick = parent_node.assigned_picks[-1] if current_depth != 1 else 0

        for i in range(highest_current_pick + 1, max_pick_number + 1):
            child_node = parent_node.get_copy()
            child_node.second_possible = False  # After Player A picks, Player B picks
            child_node.assigned_picks.append(i)
            parent_node.child_nodes.append(child_node)
            calculate_pick_combinations(child_node, territory_count)

    # Player B is picking
    if second_still_possible:
        opponent_picks = current_depth if current_depth % 2 == 1 else current_depth - 1
        max_pick_number = opponent_picks + 1 + own_picks
        highest_current_pick = parent_node.assigned_picks[-1] if current_depth != 1 else 0

        for i in range(highest_current_pick + 1, max_pick_number + 1):
            child_node = parent_node.get_copy()
            child_node.first_possible = False  # After Player B picks, Player A picks
            child_node.assigned_picks.append(i)
            parent_node.child_nodes.append(child_node)
            calculate_pick_combinations(child_node, territory_count)


def get_all_leaves(root):
    leaves = []

    if not root.child_nodes:
        leaves.append(root)
    else:
        for child in root.child_nodes:
            leaves.extend(get_all_leaves(child))

    return leaves


def print_grouped_combinations(root, top_n=2):
    leaf_nodes = get_all_leaves(root)
    groups = defaultdict(dict)

    for node in leaf_nodes:
        picks_tuple = tuple(node.assigned_picks[:top_n])  # Group by the first 'top_n' picks
        remaining_picks_tuple = tuple(node.assigned_picks)  # Use full tuple of picks

        if remaining_picks_tuple in groups[picks_tuple]:
            # Update the first/second flags
            if node.first_possible:
                groups[picks_tuple][remaining_picks_tuple].first_possible = True
            if node.second_possible:
                groups[picks_tuple][remaining_picks_tuple].second_possible = True
        else:
            # Store new combination
            groups[picks_tuple][remaining_picks_tuple] = node

    # Print each group of combinations
    for group_key, group_nodes in groups.items():
        print(f"Group based on top {top_n} picks: {group_key}")
        for node in group_nodes.values():
            print(f"  {node}")
        print()


def main(territory_count=7, top_n=2):
    root = Node()
    calculate_pick_combinations(root, territory_count)
    print_grouped_combinations(root, top_n)


if __name__ == "__main__":
    main(territory_count=7, top_n=2)


Group based on top 2 picks: (1, 2)
  [1, 2, 3, 4, 5, 6, 7] (first, second)
  [1, 2, 3, 4, 5, 6, 8] (first, second)
  [1, 2, 3, 4, 5, 6, 9] (first, second)
  [1, 2, 3, 4, 5, 6, 10] (first, second)
  [1, 2, 3, 4, 5, 6, 11] (first, second)
  [1, 2, 3, 4, 5, 6, 12] (first, second)
  [1, 2, 3, 4, 5, 6, 13] (first, second)
  [1, 2, 3, 4, 5, 7, 8] (first, second)
  [1, 2, 3, 4, 5, 7, 9] (first, second)
  [1, 2, 3, 4, 5, 7, 10] (first, second)
  [1, 2, 3, 4, 5, 7, 11] (first, second)
  [1, 2, 3, 4, 5, 7, 12] (first, second)
  [1, 2, 3, 4, 5, 7, 13] (first, second)
  [1, 2, 3, 4, 5, 8, 9] (first, second)
  [1, 2, 3, 4, 5, 8, 10] (first, second)
  [1, 2, 3, 4, 5, 8, 11] (first, second)
  [1, 2, 3, 4, 5, 8, 12] (first, second)
  [1, 2, 3, 4, 5, 8, 13] (first, second)
  [1, 2, 3, 4, 5, 9, 10] (first, second)
  [1, 2, 3, 4, 5, 9, 11] (first, second)
  [1, 2, 3, 4, 5, 9, 12] (first, second)
  [1, 2, 3, 4, 5, 9, 13] (first, second)
  [1, 2, 3, 4, 5, 10, 11] (first, second)
  [1, 2, 3, 4, 5, 10, 12] (

if __name__ == "__main__":
    main(territory_count=5, top_n=2, difference=3)

In [63]:
from collections import defaultdict

class Node:
    def __init__(self, assigned_picks=None, first_possible=True, second_possible=True):
        self.child_nodes = []
        self.assigned_picks = assigned_picks if assigned_picks is not None else []
        self.first_possible = first_possible
        self.second_possible = second_possible

    def get_copy(self):
        copy_node = Node(self.assigned_picks.copy(), self.first_possible, self.second_possible)
        return copy_node

    def __str__(self):
        picks_str = ", ".join(map(str, self.assigned_picks))
        possible_str = []
        if self.first_possible:
            possible_str.append("first")
        if self.second_possible:
            possible_str.append("second")
        return f"[{picks_str}] ({', '.join(possible_str)})"


def calculate_pick_combinations(parent_node, territory_count):
    if len(parent_node.assigned_picks) == territory_count:
        return

    first_still_possible = parent_node.first_possible
    second_still_possible = parent_node.second_possible
    current_depth = len(parent_node.assigned_picks) + 1

    own_picks = current_depth - 1

    # Player A is picking
    if first_still_possible:
        opponent_picks = current_depth - 1 if current_depth % 2 == 1 else current_depth
        max_pick_number = opponent_picks + 1 + own_picks
        highest_current_pick = parent_node.assigned_picks[-1] if current_depth != 1 else 0

        for i in range(highest_current_pick + 1, max_pick_number + 1):
            child_node = parent_node.get_copy()
            child_node.second_possible = False  # After Player A picks, Player B picks
            child_node.assigned_picks.append(i)
            parent_node.child_nodes.append(child_node)
            calculate_pick_combinations(child_node, territory_count)

    # Player B is picking
    if second_still_possible:
        opponent_picks = current_depth if current_depth % 2 == 1 else current_depth - 1
        max_pick_number = opponent_picks + 1 + own_picks
        highest_current_pick = parent_node.assigned_picks[-1] if current_depth != 1 else 0

        for i in range(highest_current_pick + 1, max_pick_number + 1):
            child_node = parent_node.get_copy()
            child_node.first_possible = False  # After Player B picks, Player A picks
            child_node.assigned_picks.append(i)
            parent_node.child_nodes.append(child_node)
            calculate_pick_combinations(child_node, territory_count)


def get_all_leaves(root):
    leaves = []

    if not root.child_nodes:
        leaves.append(root)
    else:
        for child in root.child_nodes:
            leaves.extend(get_all_leaves(child))

    return leaves


def print_grouped_combinations(root, top_n=2, difference=None):
    leaf_nodes = get_all_leaves(root)
    groups = defaultdict(dict)

    for node in leaf_nodes:
        picks_tuple = tuple(node.assigned_picks[:top_n])  # Group by the first 'top_n' picks

        # Apply the difference filter if specified
        if difference is not None:
            if len(picks_tuple) < 2 or picks_tuple[1] - picks_tuple[0] != difference:
                continue  # Skip this combination

        remaining_picks_tuple = tuple(node.assigned_picks)  # Use full tuple of picks

        if remaining_picks_tuple in groups[picks_tuple]:
            # Update the first/second flags
            if node.first_possible:
                groups[picks_tuple][remaining_picks_tuple].first_possible = True
            if node.second_possible:
                groups[picks_tuple][remaining_picks_tuple].second_possible = True
        else:
            # Store new combination
            groups[picks_tuple][remaining_picks_tuple] = node

    # Print each group of combinations
    for group_key, group_nodes in groups.items():
        print(f"Group based on top {top_n} picks: {group_key}")
        for node in group_nodes.values():
            print(f"  {node}")
        print()


def main(territory_count=5, top_n=2, difference=3):
    root = Node()
    calculate_pick_combinations(root, territory_count)
    print_grouped_combinations(root, top_n, difference)


if __name__ == "__main__":
    main(territory_count=5, top_n=2, difference=3)


Group based on top 2 picks: (1, 4)
  [1, 4, 5, 6, 7] (first)
  [1, 4, 5, 6, 8] (first)
  [1, 4, 5, 6, 9] (first)
  [1, 4, 5, 7, 8] (first)
  [1, 4, 5, 7, 9] (first)
  [1, 4, 5, 8, 9] (first)



if __name__ == "__main__":
    main(territory_count=5, difference=2, turn_indices=(0, 1))

i want to look for example for mirroring first 7 picks which means 
A -> 1/4/5/8/9
B -> 2/3/6/7
I can do "main(territory_count=9, difference=3, turn_indices=(1, 2))" -> this searches for the 2/3/6/7 difference of 3 between 3/6.

In [90]:
from collections import defaultdict

class Node:
    def __init__(self, assigned_picks=None, pick_turns=None, first_possible=True, second_possible=True):
        self.child_nodes = []
        self.assigned_picks = assigned_picks if assigned_picks is not None else []
        self.pick_turns = pick_turns if pick_turns is not None else []
        self.first_possible = first_possible
        self.second_possible = second_possible

    def get_copy(self):
        copy_node = Node(self.assigned_picks.copy(), self.pick_turns.copy(), self.first_possible, self.second_possible)
        return copy_node

    def __str__(self):
        picks_with_turns = ", ".join(f"{pick}(t{turn})" for pick, turn in zip(self.assigned_picks, self.pick_turns))
        possible_str = []
        if self.first_possible:
            possible_str.append("first")
        if self.second_possible:
            possible_str.append("second")
        return f"[{picks_with_turns}] ({', '.join(possible_str)})"

def calculate_pick_combinations(parent_node, territory_count, current_turn=1):
    if len(parent_node.assigned_picks) == territory_count:
        return

    first_still_possible = parent_node.first_possible
    second_still_possible = parent_node.second_possible
    current_depth = len(parent_node.assigned_picks) + 1

    own_picks = current_depth - 1

    # Player A is picking
    if first_still_possible:
        opponent_picks = current_depth - 1 if current_depth % 2 == 1 else current_depth
        max_pick_number = opponent_picks + 1 + own_picks
        highest_current_pick = parent_node.assigned_picks[-1] if current_depth != 1 else 0

        for i in range(highest_current_pick + 1, max_pick_number + 1):
            child_node = parent_node.get_copy()
            # Correctly set the second_possible flag to False when Player A picks
            child_node.second_possible = False
            child_node.assigned_picks.append(i)
            child_node.pick_turns.append(current_turn)
            parent_node.child_nodes.append(child_node)
            calculate_pick_combinations(child_node, territory_count, current_turn + 1)

    # Player B is picking
    if second_still_possible:
        opponent_picks = current_depth if current_depth % 2 == 1 else current_depth - 1
        max_pick_number = opponent_picks + 1 + own_picks
        highest_current_pick = parent_node.assigned_picks[-1] if current_depth != 1 else 0

        for i in range(highest_current_pick + 1, max_pick_number + 1):
            child_node = parent_node.get_copy()
            # Correctly set the first_possible flag to False when Player B picks
            child_node.first_possible = False
            child_node.assigned_picks.append(i)
            child_node.pick_turns.append(current_turn)
            parent_node.child_nodes.append(child_node)
            calculate_pick_combinations(child_node, territory_count, current_turn + 1)

def get_all_leaves(root):
    leaves = []

    if not root.child_nodes:
        leaves.append(root)
    else:
        for child in root.child_nodes:
            leaves.extend(get_all_leaves(child))

    return leaves

def print_combinations_with_difference(root, difference, turn_indices=None):
    leaf_nodes = get_all_leaves(root)
    groups = defaultdict(dict)  # groups[group_key][combination_tuple] = node

    for node in leaf_nodes:
        picks = node.assigned_picks
        turns = node.pick_turns
        # If turn_indices are specified, check difference between picks at those turns
        if turn_indices is not None:
            idx1, idx2 = turn_indices
            if idx1 < len(picks) and idx2 < len(picks):
                pick1 = picks[idx1]
                pick2 = picks[idx2]
                if pick2 - pick1 != difference:
                    continue  # Skip this node
            else:
                continue  # Skip if indices are out of bounds
        else:
            # Check all pairs of picks for the specified difference
            found_difference = False
            for i in range(len(picks)):
                for j in range(i+1, len(picks)):
                    if picks[j] - picks[i] == difference:
                        found_difference = True
                        break
                if found_difference:
                    break
            if not found_difference:
                continue  # Skip this node

        # Now group the combinations
        combination_tuple = tuple(picks)
        # Use combination_tuple as group key to ensure uniqueness
        group_key = (picks[idx1], picks[idx2]) if turn_indices else None

        if group_key is None:
            group_key = combination_tuple[:2]  # group by first two picks

        # Check if the combination is already in the group
        if combination_tuple in groups[group_key]:
            existing_node = groups[group_key][combination_tuple]
            # Update the first/second possible flags
            existing_node.first_possible = existing_node.first_possible or node.first_possible
            existing_node.second_possible = existing_node.second_possible or node.second_possible
        else:
            # Add the node to the group
            groups[group_key][combination_tuple] = node

    # Now print the groups
    for group_key, nodes_dict in groups.items():
        print(f"Group with picks having difference {difference}: {group_key}")
        for node in nodes_dict.values():
            print(f"  {node}")
        print()

def main(territory_count=5, difference=3, turn_indices=(0, 2)):
    root = Node()
    calculate_pick_combinations(root, territory_count)
    print_combinations_with_difference(root, difference, turn_indices)

if __name__ == "__main__":
    main(territory_count=5, difference=2, turn_indices=(0, 1))


Group with picks having difference 2: (1, 3)
  [1(t1), 3(t2), 4(t3), 5(t4), 6(t5)] (first, second)
  [1(t1), 3(t2), 4(t3), 5(t4), 7(t5)] (first, second)
  [1(t1), 3(t2), 4(t3), 5(t4), 8(t5)] (first, second)
  [1(t1), 3(t2), 4(t3), 5(t4), 9(t5)] (first, second)
  [1(t1), 3(t2), 4(t3), 6(t4), 7(t5)] (first, second)
  [1(t1), 3(t2), 4(t3), 6(t4), 8(t5)] (first, second)
  [1(t1), 3(t2), 4(t3), 6(t4), 9(t5)] (first, second)
  [1(t1), 3(t2), 4(t3), 7(t4), 8(t5)] (first, second)
  [1(t1), 3(t2), 4(t3), 7(t4), 9(t5)] (first, second)
  [1(t1), 3(t2), 4(t3), 8(t4), 9(t5)] (first)
  [1(t1), 3(t2), 5(t3), 6(t4), 7(t5)] (first, second)
  [1(t1), 3(t2), 5(t3), 6(t4), 8(t5)] (first, second)
  [1(t1), 3(t2), 5(t3), 6(t4), 9(t5)] (first, second)
  [1(t1), 3(t2), 5(t3), 7(t4), 8(t5)] (first, second)
  [1(t1), 3(t2), 5(t3), 7(t4), 9(t5)] (first, second)
  [1(t1), 3(t2), 5(t3), 8(t4), 9(t5)] (first)
  [1(t1), 3(t2), 4(t3), 5(t4), 10(t5)] (second)
  [1(t1), 3(t2), 4(t3), 6(t4), 10(t5)] (second)
  [1(t1), 3

filter only with pick difference

In [95]:
from collections import defaultdict

class Node:
    def __init__(self, assigned_picks=None, pick_turns=None, first_possible=True, second_possible=True):
        self.child_nodes = []
        self.assigned_picks = assigned_picks if assigned_picks is not None else []
        self.pick_turns = pick_turns if pick_turns is not None else []
        self.first_possible = first_possible
        self.second_possible = second_possible

    def get_copy(self):
        copy_node = Node(self.assigned_picks.copy(), self.pick_turns.copy(), self.first_possible, self.second_possible)
        return copy_node

    def __str__(self):
        picks_str = ", ".join(map(str, self.assigned_picks))
        possible_str = []
        if self.first_possible:
            possible_str.append("first")
        if self.second_possible:
            possible_str.append("second")
        return f"[{picks_str}] ({', '.join(possible_str)})"

def calculate_pick_combinations(parent_node, territory_count, current_turn=1):
    if len(parent_node.assigned_picks) == territory_count:
        return

    first_still_possible = parent_node.first_possible
    second_still_possible = parent_node.second_possible
    current_depth = len(parent_node.assigned_picks) + 1

    own_picks = current_depth - 1

    # Player A is picking
    if first_still_possible:
        opponent_picks = current_depth - 1 if current_depth % 2 == 1 else current_depth
        max_pick_number = opponent_picks + 1 + own_picks
        highest_current_pick = parent_node.assigned_picks[-1] if current_depth != 1 else 0

        for i in range(highest_current_pick + 1, max_pick_number + 1):
            child_node = parent_node.get_copy()
            child_node.second_possible = False  # After Player A picks, Player B picks
            child_node.assigned_picks.append(i)
            parent_node.child_nodes.append(child_node)
            calculate_pick_combinations(child_node, territory_count, current_turn + 1)

    # Player B is picking
    if second_still_possible:
        opponent_picks = current_depth if current_depth % 2 == 1 else current_depth - 1
        max_pick_number = opponent_picks + 1 + own_picks
        highest_current_pick = parent_node.assigned_picks[-1] if current_depth != 1 else 0

        for i in range(highest_current_pick + 1, max_pick_number + 1):
            child_node = parent_node.get_copy()
            child_node.first_possible = False  # After Player B picks, Player A picks
            child_node.assigned_picks.append(i)
            parent_node.child_nodes.append(child_node)
            calculate_pick_combinations(child_node, territory_count, current_turn + 1)

def get_all_leaves(root):
    leaves = []

    if not root.child_nodes:
        leaves.append(root)
    else:
        for child in root.child_nodes:
            leaves.extend(get_all_leaves(child))

    return leaves

def print_combinations_with_consecutive_difference(root, difference):
    leaf_nodes = get_all_leaves(root)
    combinations_found = []

    for node in leaf_nodes:
        picks = node.assigned_picks
        # Check consecutive picks for the specified difference
        found_difference = False
        for i in range(len(picks) - 1):
            if picks[i + 1] - picks[i] == difference:
                found_difference = True
                break
        if found_difference:
            combinations_found.append(node)

    # Group combinations to avoid duplicates
    combinations_dict = {}
    for node in combinations_found:
        picks_tuple = tuple(node.assigned_picks)
        if picks_tuple in combinations_dict:
            existing_node = combinations_dict[picks_tuple]
            existing_node.first_possible |= node.first_possible
            existing_node.second_possible |= node.second_possible
        else:
            combinations_dict[picks_tuple] = node

    # Print the combinations
    for node in combinations_dict.values():
        print(node)

def main(territory_count=5, difference=4):
    root = Node()
    calculate_pick_combinations(root, territory_count)
    print_combinations_with_consecutive_difference(root, difference)

if __name__ == "__main__":
    main(territory_count=5, difference=5)


[1, 2, 3, 4, 9] (first, second)
[1, 2, 3, 8, 9] (first)
[1, 2, 3, 5, 10] (second)
[1, 2, 4, 5, 10] (second)
[1, 3, 4, 5, 10] (second)
[2, 3, 4, 5, 10] (second)


checking whether or not i can lose a range of picks. example: |main(territory_count=5, excluded_picks=[3, 4, 5, 6])|

checking for unicorn island whether I can lose all 3,4,5,6 at the same time in any combination -> I can't

In [115]:
from collections import defaultdict

class Node:
    def __init__(self, assigned_picks=None, first_possible=True, second_possible=True):
        self.child_nodes = []
        self.assigned_picks = assigned_picks if assigned_picks is not None else []
        self.first_possible = first_possible
        self.second_possible = second_possible

    def get_copy(self):
        copy_node = Node(self.assigned_picks.copy(), self.first_possible, self.second_possible)
        return copy_node

    def __str__(self):
        picks_str = ", ".join(map(str, self.assigned_picks))
        possible_str = []
        if self.first_possible:
            possible_str.append("first")
        if self.second_possible:
            possible_str.append("second")
        return f"[{picks_str}] ({', '.join(possible_str)})"

def calculate_pick_combinations(parent_node, territory_count):
    if len(parent_node.assigned_picks) == territory_count:
        return

    first_still_possible = parent_node.first_possible
    second_still_possible = parent_node.second_possible
    current_depth = len(parent_node.assigned_picks) + 1

    own_picks = current_depth - 1

    # Player A is picking
    if first_still_possible:
        opponent_picks = current_depth - 1 if current_depth % 2 == 1 else current_depth
        max_pick_number = opponent_picks + 1 + own_picks
        highest_current_pick = parent_node.assigned_picks[-1] if current_depth != 1 else 0

        for i in range(highest_current_pick + 1, max_pick_number + 1):
            child_node = parent_node.get_copy()
            child_node.second_possible = False  # After Player A picks, Player B picks
            child_node.assigned_picks.append(i)
            parent_node.child_nodes.append(child_node)
            calculate_pick_combinations(child_node, territory_count)

    # Player B is picking
    if second_still_possible:
        opponent_picks = current_depth if current_depth % 2 == 1 else current_depth - 1
        max_pick_number = opponent_picks + 1 + own_picks
        highest_current_pick = parent_node.assigned_picks[-1] if current_depth != 1 else 0

        for i in range(highest_current_pick + 1, max_pick_number + 1):
            child_node = parent_node.get_copy()
            child_node.first_possible = False  # After Player B picks, Player A picks
            child_node.assigned_picks.append(i)
            parent_node.child_nodes.append(child_node)
            calculate_pick_combinations(child_node, territory_count)

def get_all_leaves(root):
    leaves = []

    if not root.child_nodes:
        leaves.append(root)
    else:
        for child in root.child_nodes:
            leaves.extend(get_all_leaves(child))

    return leaves

def print_filtered_combinations(root, excluded_picks):
    leaf_nodes = get_all_leaves(root)
    combinations_dict = {}

    for node in leaf_nodes:
        picks = node.assigned_picks

        # Check if any pick is in the excluded_picks set
        if any(pick in excluded_picks for pick in picks):
            continue  # Skip this combination

        # Group combinations to avoid duplicates
        picks_tuple = tuple(picks)
        if picks_tuple in combinations_dict:
            existing_node = combinations_dict[picks_tuple]
            existing_node.first_possible |= node.first_possible
            existing_node.second_possible |= node.second_possible
        else:
            combinations_dict[picks_tuple] = node

    # Print the combinations
    for node in combinations_dict.values():
        print(node)

def main(territory_count=5, excluded_picks=None):
    if excluded_picks is None:
        excluded_picks = set()
    else:
        excluded_picks = set(excluded_picks)

    root = Node()
    calculate_pick_combinations(root, territory_count)
    print_filtered_combinations(root, excluded_picks)

if __name__ == "__main__":
    main(territory_count=5, excluded_picks=[4,5,6])


[1, 2, 3, 7, 8] (first, second)
[1, 2, 3, 7, 9] (first, second)
[1, 2, 3, 8, 9] (first)
[1, 2, 3, 7, 10] (second)


finding pick combinations with
- excluding certain picks
- including certain picks

In [105]:
from collections import defaultdict

class Node:
    def __init__(self, assigned_picks=None, first_possible=True, second_possible=True):
        self.child_nodes = []
        self.assigned_picks = assigned_picks if assigned_picks is not None else []
        self.first_possible = first_possible
        self.second_possible = second_possible

    def get_copy(self):
        copy_node = Node(self.assigned_picks.copy(), self.first_possible, self.second_possible)
        return copy_node

    def __str__(self):
        picks_str = ", ".join(map(str, self.assigned_picks))
        possible_str = []
        if self.first_possible:
            possible_str.append("first")
        if self.second_possible:
            possible_str.append("second")
        return f"[{picks_str}] ({', '.join(possible_str)})"

def calculate_pick_combinations(parent_node, territory_count):
    if len(parent_node.assigned_picks) == territory_count:
        return

    first_still_possible = parent_node.first_possible
    second_still_possible = parent_node.second_possible
    current_depth = len(parent_node.assigned_picks) + 1

    own_picks = current_depth - 1

    # Player A is picking
    if first_still_possible:
        opponent_picks = current_depth - 1 if current_depth % 2 == 1 else current_depth
        max_pick_number = opponent_picks + 1 + own_picks
        highest_current_pick = parent_node.assigned_picks[-1] if current_depth != 1 else 0

        for i in range(highest_current_pick + 1, max_pick_number + 1):
            child_node = parent_node.get_copy()
            child_node.second_possible = False  # After Player A picks, Player B picks
            child_node.assigned_picks.append(i)
            parent_node.child_nodes.append(child_node)
            calculate_pick_combinations(child_node, territory_count)

    # Player B is picking
    if second_still_possible:
        opponent_picks = current_depth if current_depth % 2 == 1 else current_depth - 1
        max_pick_number = opponent_picks + 1 + own_picks
        highest_current_pick = parent_node.assigned_picks[-1] if current_depth != 1 else 0

        for i in range(highest_current_pick + 1, max_pick_number + 1):
            child_node = parent_node.get_copy()
            child_node.first_possible = False  # After Player B picks, Player A picks
            child_node.assigned_picks.append(i)
            parent_node.child_nodes.append(child_node)
            calculate_pick_combinations(child_node, territory_count)

def get_all_leaves(root):
    leaves = []

    if not root.child_nodes:
        leaves.append(root)
    else:
        for child in root.child_nodes:
            leaves.extend(get_all_leaves(child))

    return leaves

def print_filtered_combinations(root, included_picks=None, excluded_picks=None):
    leaf_nodes = get_all_leaves(root)
    combinations_dict = {}

    if included_picks is None:
        included_picks = set()
    else:
        included_picks = set(included_picks)

    if excluded_picks is None:
        excluded_picks = set()
    else:
        excluded_picks = set(excluded_picks)

    for node in leaf_nodes:
        picks = node.assigned_picks

        # Exclude combinations that contain any of the excluded picks
        if any(pick in excluded_picks for pick in picks):
            continue  # Skip this combination

        # Exclude combinations that do not contain all of the included picks
        if not included_picks.issubset(set(picks)):
            continue  # Skip this combination

        # Group combinations to avoid duplicates
        picks_tuple = tuple(picks)
        if picks_tuple in combinations_dict:
            existing_node = combinations_dict[picks_tuple]
            existing_node.first_possible |= node.first_possible
            existing_node.second_possible |= node.second_possible
        else:
            combinations_dict[picks_tuple] = node

    # Print the combinations
    for node in combinations_dict.values():
        print(node)

def main(territory_count=5, included_picks=None, excluded_picks=None):
    root = Node()
    calculate_pick_combinations(root, territory_count)
    print_filtered_combinations(root, included_picks, excluded_picks)

if __name__ == "__main__":
    main(territory_count=5, included_picks=[1], excluded_picks=[2, 4, 5])


[1, 3, 6, 7, 8] (second)
[1, 3, 6, 7, 9] (second)
[1, 3, 6, 7, 10] (second)
